# TP : Implémentation d’un LSTM pour la génération de texte

## Introduction
Dans ce TP, vous allez mettre en œuvre un modèle de langage basé sur LSTM capable de générer du texte. 
Les étapes de ce projet sont : 
- Préparation des données, 
- Construction du vocabulaire,
- Conception et Entraînement d’un modèle séquentiel (LSTM)
- Test et Evaluation de ses performances.

Corpus d'exemple en anglais et en français, de petite et grande taille, sont fourni pour expérimenter.

### General Pipeline

1. Préparer et tokeniser un corpus textuel pour un modèle séquentiel.
2. Construire un vocabulaire mot par mot avec des tokens spéciaux (`<pad>`, `<unk>`, `<bos>`, `<eos>`).
    - `unk_token`: the unknown token.
    - `bos_token`: the beginning of sequence token.
    - `eos_token`: the end of sequence token.
    - `pad_token`: token to use for padding.
4. Créer un `Dataset` PyTorch pour générer des séquences mot par mot.
5. Implémenter et entraîner un modèle LSTM pour prédire le mot suivant.
6. Générer du texte en utilisant différentes stratégies (`greedy`, `sampling`).
7. Évaluer qualitativement et quantitativement le modèle (`perplexité`, `cohérence`, `diversité`).

## 1. Environnement et bibliothèques
Nous utiliserons `numpy`, `matplotlib`, `torch` (PyTorch) et `torch.utils.data`.
Assurez-vous d’exécuter ce notebook dans un environnement avec PyTorch.

In [ ]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

## 2. Jeux de données d'exemple
Nous avons quatre corpus pour expérimenter :
- Small English (~200-400 mots)
- Large English (~10k mots)
- Small French (~200-400 mots)
- Large French (~10k mots)

Vous pouvez remplacer ces textes par vos propres corpus.

In [ ]:
# Machiavelli's The Prince (1513)
small_english = '''
You must know there are two ways of contesting, the one by the law, the other by force; the first method is proper to men, the second to beasts; but because the first is frequently not sufficient, it is necessary to have recourse to the second. Therefore it is necessary for a prince to understand how to avail himself of the beast and the man.
'''.strip()

large_english = ' '.join([small_english]*200)

#Le Prince de Nicolas Machiavel: CHAPITRE XVIII.
small_french = '''
On peut combattre de deux manières: ou avec les lois, ou avec la force. La première est propre à l'homme, la seconde est celle des bêtes; mais comme souvent celle-là ne suffit point, on est obligé de recourir à l'autre: il faut donc qu'un prince sache agir à propos, et en bête et en homme.
'''.strip()

large_french = ' '.join([small_french]*200)

datasets = {
    'eng_small': small_english,
    'eng_large': large_english,
    'fr_small': small_french,
    'fr_large': large_french
}

for name, txt in datasets.items():
    words = txt.split()
    print(f'{name}: {len(words)} words, sample= ' + ' '.join(words[:12]) + ('...' if len(words)>12 else ''))



## 3. Prétraitement et création du vocabulaire
- Tokenisation simple mot par mot.
- Construction du vocabulaire avec tokens spéciaux.`<pad>`, `<unk>`, `<bos>`, `<eos>`

In [ ]:
def tokenize(text):
    
    ponctuations = ['.', ',', ';', ':', '!', '?']
    
    all_words = []
    
    for word in text.split():
        word = word.lower()
        for p in ponctuations:
            word = word.replace(p,'')
        all_words.append(word)
    
    return all_words

class Vocabulary:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.vocab_size = 0
        
    def build_vocab(self, texts):
        self.all_words = set()
        
        for text in texts:
            words = tokenize(text)
            self.all_words.update(words)
        self.all_words = sorted(list(self.all_words))
        
        self.word2idx['<pad>'] = 0
        self.idx2word[0] = '<pad>'
        self.word2idx['<unk>'] = 1
        self.idx2word[1] = '<unk>'
        self.word2idx['<bos>'] = 2
        self.idx2word[2] = '<bos>'
        self.word2idx['<eos>'] = 3
        self.idx2word[3] = '<eos>'
        
        for i, word in enumerate(self.all_words, start=4):
            self.word2idx.update({word:i})
            self.idx2word.update({i:word})

        self.vocab_size = len(self.all_words) + 4

    def encode(self, word):
        if word not in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]
    
    def decode(self, idx):
        if idx not in self.idx2word:
            return '<unk>'
        return self.idx2word[idx]

## 4. Dataset PyTorch 

Construire des séquences d'entrée de longueur `seq_len` contenant des indices de mots, et la target est le mot suivant (`seq_len+1`). Les batches seront de forme `(batch_size, seq_len)`.


In [ ]:
"""
class TextDataset(Dataset):
    def __init__(self, text, vocab, seq_len):
        self.vocab = vocab
        self.text = text
        self.seq_len = seq_len
        self.words = ['<bos>'] + tokenize(text) + ['<eos>']
        self.encoded_words = [self.vocab.encode(word) for word in self.words]

    def __len__(self):
        return len(self.encoded_words) - self.seq_len
    
    def __getitem__(self, idx):
        input = self.encoded_words[idx : idx + self.seq_len]
        target = self.encoded_words[idx + self.seq_len]
        return torch.tensor(input), torch.tensor(target)
"""
class TextDataset(Dataset):
    def __init__(self, sentences, vocab, seq_len):
        self.vocab = vocab
        self.seq_len = seq_len
        self.pairs = []

        for sentence in sentences:
            words = ['<bos>'] + tokenize(sentence)  + ['<eos>']
            encoded_words = []
            for word in words:
                idx = self.vocab.encode(word)
                encoded_words.append(idx)

            nb_pairs = len(encoded_words) - self.seq_len

            for i in range(nb_pairs): 
                input_seq = encoded_words[i : i + self.seq_len]
                target_seq = encoded_words[i + self.seq_len]
                self.pairs.append((input_seq, target_seq))
                
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        input_seq, target_seq = self.pairs[idx]
        return torch.tensor(input_seq), torch.tensor(target_seq)

## 5. Modèle LSTM pour la génération de texte

Vous pouvez proposer votre propre architecture ou vous inspirer de l’architecture suivante : 
- `nn.Embedding(vocab_size, embed_dim)` : convertit indices → vecteurs.
- `nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)` : traite la séquence.
- `nn.Linear(hidden_dim, vocab_size)` : prédiction du mot suivant à chaque pas de temps (nous utilisons la sortie du dernier pas pour prédire le mot suivant du segment).
- On ajoutera `Dropout` et une petite couche fully‑connected optionnelle.

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.dropout = nn.Dropout(p=0.5)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embed(x)
        embedded = self.dropout(embedded)
        #output, hidden = self.lstm(embedded, hidden)
        output, _ = self.lstm(embedded)
        output = self.dropout(output)
        linear = self.linear(output[:, -1, : ])

        return linear

## 6. Fonctions utilitaires

- **Perplexity**: la perplexité est une métrique courante pour évaluer des modèles de langage : `perplexity = exp(loss)` où `loss` est la cross‑entropy moyenne par mot.
- **Génération**: méthode de génération du texte mot par mot (greedy / sampling) : on propose deux modes
    * `greedy` (choix du mot le plus probable)
    * `sampling` (échantillonnage multinomial contrôlé par `temperature`).

In [ ]:
def perplexity(loss):
    return math.exp(loss)

def generate_words(model, vocab, prompt='<bos>', max_length=20, mode='greedy', temp = 1.0, seq_len=5):
    model.eval()
    
    tokens = [vocab.encode(prompt)]
    words = []

    for _ in range(max_length):
        recent_tokens = tokens[-seq_len:]

        if len(recent_tokens) < seq_len:
            pad_idx = vocab.encode('<pad>')
            padding = [pad_idx] * (seq_len - len(recent_tokens))
            recent_tokens = padding + recent_tokens
            
        x = torch.tensor([recent_tokens])
        logits = model(x)
    
        match mode :
            case "greedy":
                next_token = torch.argmax(logits).item()
            case "sampling":
                probs = torch.softmax(logits/temp, dim = 1)
                next_token = torch.multinomial(probs, num_samples=1).item()
    
        tokens.append(next_token)
    
        if next_token == vocab.encode('<eos>'):
            break
    
    for token in tokens:
        word = vocab.decode(token)
        words.append(word)

    return words
        

## 7. Boucle d'entraînement générique

La boucle d'entraînement :
- Définit les paramètres et cycles d'entraînement.
- Est applicable à tout jeu de données.
- Calcule la perplexité moyenne. 

In [ ]:
def train(model, dataset, epochs=10, batch_size=32, lr=0.001):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f} - Perplexité: {perplexity(avg_loss):.2f}")

## 8. Expérimentation

- Pour lancer une vraie expérience sur des grands dataset (exemple : `eng_large` ou `fr_large`), reconstruisez le vocabulaire avec ces textes, augmentez `embed_dim`, `hidden_dim`, `num_layers`, et le nombre d'époques.
- Pour des textes plus longs par génération, augmentez `length` dans `generate_words`.
- Pour réduire l'overfitting sur petits datasets : dropout plus élevé, régularisation, early stopping.

## 9. Suggestions et améliorations 

- Utiliser des tokenizers plus robustes (BPETokenizer, spaCy) et traiter la casse/ponctuation.
- Remplacer LSTM par Transformer pour de meilleurs résultats sur grands corpora.
- Sauvegarder / charger des checkpoints (`torch.save` / `torch.load`).

In [ ]:
#chargement de phrases à partir d'un fichier
def load_sentences(file_path, num_sentences):
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= num_sentences:
                break
            parts = line.strip().split('\t')
            if len(parts) >= 3:
                sentences.append(parts[2])
    return sentences

#Charger les phrases
sentences = load_sentences('fra_sentences.tsv', num_sentences=50000)

print(f"Nombre de phrases chargées : {len(sentences)}")
print(f"Exemples de phrases :")
for i in range(10):
    print(f" {i+1}. {sentences[i]}")

In [ ]:
#Test sur dataset fra_sentences.tsv de tatoeba
# 1. Vocabulaire
vocab = Vocabulary()
vocab.build_vocab(sentences)

# 2. Dataset
dataset = TextDataset(sentences, vocab, seq_len=5)

# 3. Modèle
model = LSTMModel(
    vocab_size=vocab.vocab_size,
    embed_dim=128,
    hidden_dim=256,
    num_layers=2
)

# 4. Entraîner
train(model, dataset, epochs=100, batch_size=64, lr=0.001)

# 5. Générer
#print(f"Greedy : {generate_words(model, vocab, mode='greedy', max_length=30)}")
print(f"Greedy : {generate_words(model, vocab, prompt='Je',mode='greedy', max_length=30)}")

#print(f"Sampling : {generate_words(model, vocab, mode='sampling', temp=0.8, max_length=30)}")
print(f"Sampling : {generate_words(model, vocab, prompt='Je',mode='sampling', temp=0.8, max_length=30)}")


# Annexe:

## 1. Small English Datasets

| Dataset                 | Size    | Description / Domain                                                                  | Link                                                                                                                                                                                  |
| ----------------------- | ------- | ------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Tiny Shakespeare**    | ~1 MB   | Shakespeare’s plays (~100k characters). Great for character-level LSTM experiments.   | [https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt) |
| **Alice in Wonderland** | ~160 KB | Classic novel by Lewis Carroll. Good for story-style text generation.                 | [Project Gutenberg](https://www.gutenberg.org/ebooks/11)                                                                                                                               |
| **BBC News Summary**    | ~5 MB   | Short news articles, varied topics. Useful for sentence-level generation experiments. | [Kaggle](https://www.kaggle.com/datasets/pariza/bbc-news-summary)                                                                                                                      |

## 2. Small French Datasets

| Dataset                                 | Size                | Description / Domain                                                                  | Link                                                                                                         |
| --------------------------------------- | ------------------- | ------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------ |
| **Le Petit Prince**                     | ~50 KB              | Classic French children’s novel. Ideal for simple French text generation experiments. | [Project Gutenberg FR](https://www.gutenberg.org/ebooks/12910)                                               |
| **88milSMS (sample)**                   | ~ 1,000 SMS (~50 KB) | Informal French text messages. Good for casual/conversational style experiments.      | [Sample download](https://github.com/laurentprudhon/frenchtext)                                              |
| **French News Articles (small sample)** | ~200 KB             | Short news articles from French news outlets.                                         | Can be sampled from [frenchtext](https://laurentprudhon.github.io/frenchtext/datasets/index.html) |
